In [1]:
import fundamentalanalysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime


%matplotlib inline  

In [2]:
# api_key = '00ef9804fcde0edd93b1b4821ee2f06a'
api_key = 'c81352430e2fe3c941faf0814227562b'

data

In [3]:
''' Колонки '''

tickers_list =  ['LOW', 'ACN', 'SNPS']
# ['LRCX', 'BKNG','KEY', 'BAC', 'LVS', 'UAL', 'HOG', 'CFG', 'ALK', 'MAS', 'BBY', 'EMR', 'PSX', 'MS', 'LYB', 'RTX', 'RL', 'COP', 'AMAT', 'QRVO', 'SPG', 'DRI', 'BA', 'META', 'NVDA', 'FDX', 'ADSK', 'ABMD', 'GS',
# 'GPS', 'XRX', 'BKR', 'PARA', 'FCX', 'ALK', 'WDC', 'C', 'MOS', 'OKE', 'DHI', 'BBY', 'MPC', 'AMD', 'PRU', 'EXPE', 'EOG', 'VLO', 'MAR', 'HCA', 'ALB']
# ['F', 'WU', 'KMI', 'CNP', 'HPQ', 'GLW', 'KHC', 'LUV', 'CMCSA', 'NRG', 'BK', 'CSCO', 'USB', 'TJX', 'CAH', 'HIG', 'SCHW', 'HAS', 'WELL', 'DXCM', 'ROST', 'GRMN', 'NKE', 'YUM', 'GOOGL', 'JPM', 'GOOG', 'GPN', 'MMM', 'AXP', 'TGT', 'TXN', 'CRM', 'NDAQ', 'CAT', 'ILMN', 'V', 'GD', 'NFLX', 'MSFT', 'HD', 'MA', 'SPGI', 'IDXX', 'ADBE', 'AVGO',
# 'UAA', 'HBAN', 'HPE', 'IVZ', 'KEY', 'RF', 'BEN', 'WMB', 'DAL', 'GM', 'WRK', 'WFC', 'EBAY', 'VTR', 'AIG', 'MU', 'MET', 'NTAP', 'SYY', 'BXP', 'MNST', 'XOM', 'PYPL', 'DIS', 'TROW', 'HLT', 'QCOM', 'CVX', 'AAPL', 'HON', 'UPS']
# ['ADBE', 'COP', 'AMAT', 'QRVO', 'SPG', 'DRI', 'BA', 'META', 'NVDA', 'FDX', 'ADSK', 'ABMD', 'GS']


required_year = '2022'

''' время '''
stime='1999-01-01'
period = '1mo'  # '1d' '1wk'

In [4]:
def one_stock_analyse(ticker, api_key, required_year):

    def get_data_from_ticker(tick, interval='1mo'):
        ticker = yf.Ticker(tick)

        df = ticker.history(start='2017-01-01', end=None, interval=interval)
        x = pd.DataFrame(df)
        x.rename(columns={"Close": tick}, inplace=True)
        z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
        return z

    df = get_data_from_ticker(ticker)



    def change_index(df):
        df_index = df.index.to_list()
        for i in range(len(df_index)):
            t = df_index[i].date()
            df_index[i] = t.strftime('%Y-%m-%d')
        df['Date_index'] = df_index
        df = df.set_index('Date_index')
        return df

    df = change_index(df)


    def get_key_metricks(ticker, api_key, required_year):
        key_metrics = fa.key_metrics(ticker, api_key, period='annual')
        df_key_metr = key_metrics.T
        df_key_metr = df_key_metr.loc[required_year:'2000'].copy(deep=True)
        return df_key_metr

    
    
    df_key_metr = get_key_metricks(ticker, api_key, required_year)
    
    
    if df_key_metr.index[0] == required_year and f'{int(required_year)+1}-01-01' in df.index.to_list():
        last_year = True
    else:
        last_year = False



    def get_index(df):
        index_years = df.index.to_list()
        return index_years

    index_years = get_index(df_key_metr)



    def get_price_from_df(df, list_dates):
        new_list = []
        for i in range(len(list_dates)):
            x = int(list_dates[i])
            x += 1
            x = str(x)
            try:
                r = df.loc[f'{x}-01-01']
                r = float(r)
                new_list.append(r)
            except:
                new_list.append(0)
                
        return new_list

    stock_year_prices = get_price_from_df(df, index_years)


    df_key_metr[ticker] = list(stock_year_prices)
    df_key_metr.drop('period', axis=1, inplace=True)

    def change_percent_all_columns(df):
        x = df.columns.to_list()
        for i in range(len(x)):
            v = df[x[i]].to_list()
            l = []
            for y in range(len(v)):
                try:
                    if y != len(v):
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv)
                    else:
                        vv = 0
                        l.append(vv)
                except:
                    l.append(0)
            df[f'change % {x[i]}'] = l
        return df
    
    df_with_changes = change_percent_all_columns(df_key_metr)
    
    if last_year == True:
        # df_last = df_with_changes.reindex(index=df_with_changes.index[::-1])
        r = [ticker, df_with_changes[f'change % {ticker}'][0], df_with_changes['change % debtToAssets'][0]]
    else:
        r = None 
        
        
    
    return r, print(r)

results of stock's data

In [5]:
def get_results_da_price(tickers_list, api_key, required_year):
    l = []
    for i in range(len(tickers_list)):
        r = one_stock_analyse(tickers_list[i], api_key, required_year)
        if r == None:
            pass
        else:
            l.append(r)
    return l
  
df = pd.DataFrame(get_results_da_price(tickers_list, api_key, required_year), columns = ['Ticker', 'Change % price', 'change % debtToAssets'])
df

['LOW', -10.084017215071896, 14.303103256792042]
['ACN', -17.996593347804435, -2.9581578159273363]
['SNPS', 8.631236004176728, 3.844680099680616]


ValueError: 3 columns passed, passed data had 2 columns

In [ ]:
l  = []
for i in range(len(df['Change % price'])):
    if df['Change % price'][i] > 0 and df['change % debtToAssets'][i] < 0:
        l.append(True)
    elif df['Change % price'][i] < 0 and df['change % debtToAssets'][i] > 0:
        l.append(True)
    else:
        l.append(False)

df['Conformity'] = l
df

In [ ]:
l  = []
for i in range(len(df['Change % price'])):
    if df['Change % price'][i] > 0 and df['change % debtToAssets'][i] < 0:
        r = df['Change % price'][i] + df['change % debtToAssets'][i]
        r = r *(-1) if df['Change % price'][i] > df['change % debtToAssets'][i] * -1 else r
        l.append(r)
    elif df['Change % price'][i] < 0 and df['change % debtToAssets'][i] > 0:
        r = df['Change % price'][i] + df['change % debtToAssets'][i]
        r = r *(-1) if df['Change % price'][i] * -1 > df['change % debtToAssets'][i] else r
        l.append(r)
    else:
        l.append(0)

df['Differense'] = l


results

In [ ]:
df